In [12]:
import yfinance as yf
from bs4 import BeautifulSoup
import httpx
import pandas as pd
import numpy as np
import duckdb as db
from datetime import datetime


In [13]:
stocks=pd.read_csv("../../data/ind_nifty500list.csv")
stocks=stocks[["Symbol","Industry"]]

In [27]:
stocks

,Symbol,Industry
0,360ONE,Financial Services
1,3MINDIA,Diversified
2,ABB,Capital Goods
3,ACC,Construction Materials
4,ACMESOLAR,Power
...,...,...
496,ZEEL,Media Entertainment & Publication
497,ZENTEC,Capital Goods
498,ZENSARTECH,Information Technology
499,ZYDUSLIFE,Healthcare


In [15]:
stocks["Industry"].value_counts()

Industry
Financial Services                   95
Capital Goods                        64
Healthcare                           50
Automobile and Auto Components       33
Fast Moving Consumer Goods           30
Information Technology               29
Chemicals                            27
Consumer Services                    25
Consumer Durables                    19
Oil Gas & Consumable Fuels           18
Metals & Mining                      17
Power                                17
Services                             15
Construction                         15
Realty                               11
Telecommunication                    11
Construction Materials                9
Textiles                              7
Media Entertainment & Publication     5
Diversified                           3
Forest Materials                      1
Name: count, dtype: int64

In [16]:
stocks["Industry"].nunique()

21

In [28]:
page = httpx.get("https://www.screener.in/company/3MINDIA/consolidated/#quarters")
soup = BeautifulSoup(page.text, "html.parser")

In [29]:
table = soup.find("section", {"id": "quarters"}).find("table")

In [30]:
table

<table class="data-table responsive-text-nowrap">
<thead>
<tr>
<th class="text"></th>
<th class="highlight-cell">
            Jun 2021
            
          </th>
<th class="">
            Sep 2021
            
          </th>
<th class="">
            Dec 2021
            
          </th>
<th class="">
            Mar 2022
            
          </th>
<th class="highlight-cell">
            Jun 2022
            
          </th>
<th class="">
            Sep 2022
            
          </th>
<th class="">
            Dec 2022
            
          </th>
<th class="">
            Mar 2023
            
          </th>
<th class="highlight-cell">
            Jun 2023
            
          </th>
<th class="">
            Sep 2023
            
          </th>
<th class="">
            Dec 2023
            
          </th>
<th class="">
            Mar 2024
            
          </th>
<th class="highlight-cell">
            Jun 2024
            
          </th>
</tr>
</thead>
<tbody>
<tr

In [32]:
headers = [datetime.strptime(th.get_text(strip=True), "%b %Y").strftime("%Y-%m") for th in table.find("thead").find_all("th")[1:]]

# Extract rows
rows = table.find("tbody").find_all("tr")

# Initialize dictionary
data = {header: {} for header in headers}


# Iterate over rows and populate data
for row in rows:
    cols = row.find_all("td")
    row_label = cols[0].get_text(strip=True).replace("+", "")
    for header, col in zip(headers, cols[1:]):
        value = col.get_text(strip=True).replace(',', '').replace('%', '')
        try:
            value = float(value) if '.' in value else int(value)
        except ValueError:
            value = None
        data[header][row_label] = value

print(data)

{'2021-06': {'Sales': 744, 'Expenses': 696, 'Operating Profit': 48, 'OPM %': 6, 'Other Income': 8, 'Interest': 1, 'Depreciation': 15, 'Profit before tax': 41, 'Tax %': 26, 'Net Profit': 30, 'EPS in Rs': 26.72, 'Raw PDF': None}, '2021-09': {'Sales': 840, 'Expenses': 748, 'Operating Profit': 92, 'OPM %': 11, 'Other Income': 8, 'Interest': 1, 'Depreciation': 13, 'Profit before tax': 87, 'Tax %': 26, 'Net Profit': 64, 'EPS in Rs': 57.01, 'Raw PDF': None}, '2021-12': {'Sales': 826, 'Expenses': 731, 'Operating Profit': 95, 'OPM %': 12, 'Other Income': 8, 'Interest': 0, 'Depreciation': 14, 'Profit before tax': 89, 'Tax %': 25, 'Net Profit': 67, 'EPS in Rs': 59.24, 'Raw PDF': None}, '2022-03': {'Sales': 926, 'Expenses': 768, 'Operating Profit': 158, 'OPM %': 17, 'Other Income': 9, 'Interest': 2, 'Depreciation': 13, 'Profit before tax': 151, 'Tax %': 27, 'Net Profit': 111, 'EPS in Rs': 98.46, 'Raw PDF': None}, '2022-06': {'Sales': 944, 'Expenses': 827, 'Operating Profit': 116, 'OPM %': 12, 'Oth

In [ ]:
# convert Dec 2021 to 2021-12 using strptime

In [33]:
data

{'2021-06': {'Sales': 744,
  'Expenses': 696,
  'Operating Profit': 48,
  'OPM %': 6,
  'Other Income': 8,
  'Interest': 1,
  'Depreciation': 15,
  'Profit before tax': 41,
  'Tax %': 26,
  'Net Profit': 30,
  'EPS in Rs': 26.72,
  'Raw PDF': None},
 '2021-09': {'Sales': 840,
  'Expenses': 748,
  'Operating Profit': 92,
  'OPM %': 11,
  'Other Income': 8,
  'Interest': 1,
  'Depreciation': 13,
  'Profit before tax': 87,
  'Tax %': 26,
  'Net Profit': 64,
  'EPS in Rs': 57.01,
  'Raw PDF': None},
 '2021-12': {'Sales': 826,
  'Expenses': 731,
  'Operating Profit': 95,
  'OPM %': 12,
  'Other Income': 8,
  'Interest': 0,
  'Depreciation': 14,
  'Profit before tax': 89,
  'Tax %': 25,
  'Net Profit': 67,
  'EPS in Rs': 59.24,
  'Raw PDF': None},
 '2022-03': {'Sales': 926,
  'Expenses': 768,
  'Operating Profit': 158,
  'OPM %': 17,
  'Other Income': 9,
  'Interest': 2,
  'Depreciation': 13,
  'Profit before tax': 151,
  'Tax %': 27,
  'Net Profit': 111,
  'EPS in Rs': 98.46,
  'Raw PDF': N

In [36]:
temp=pd.DataFrame(data).T

In [39]:
temp.pct_change().round(3)

/var/folders/jf/1r7jb1_s6ys1p0n0n56s2qsh0000gn/T/ipykernel_5993/2850249356.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  temp.pct_change().round(3)


,Sales,Expenses,Operating Profit,OPM %,Other Income,Interest,Depreciation,Profit before tax,Tax %,Net Profit,EPS in Rs,Raw PDF
2021-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-09,0.129,0.075,0.917,0.833,0.000,0.000,-0.133,1.122,0.000,1.133,1.134,NaN
2021-12,-0.017,-0.023,0.033,0.091,0.000,-1.000,0.077,0.023,-0.038,0.047,0.039,NaN
2022-03,0.121,0.051,0.663,0.417,0.125,inf,-0.071,0.697,0.080,0.657,0.662,NaN
2022-06,0.019,0.077,-0.266,-0.294,0.222,-0.500,0.077,-0.252,-0.037,-0.243,-0.240,NaN
2022-09,0.035,0.033,0.060,0.083,2.182,-1.000,0.071,0.265,0.000,0.262,0.261,NaN
2022-12,0.016,-0.037,0.390,0.308,-0.657,inf,0.000,0.168,-0.038,0.179,0.175,NaN
2023-03,0.053,0.036,0.135,0.118,-0.167,5.000,-0.067,0.102,0.040,0.088,0.087,NaN
2023-06,0.004,0.029,-0.108,-0.158,0.500,-0.833,0.000,-0.060,-0.038,-0.051,-0.048,NaN
2023-09,-0.010,-0.034,0.110,0.125,0.133,0.000,-0.071,0.133,0.000,0.132,0.131,NaN
